**Data Cleaning for the schools dataset** so that we have the number of schools in each region and suburb of Melbourne

In [49]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [50]:
#Reading the csv school data file 
school_data = pd.read_csv(r'C:\Users\DELL\OneDrive\Documents\GitHub\HEM-Housing-ML-Model\Datasets\School.csv')
school_data.head()

,Address_Town,Education_Sector,School_Type,LGA_Name,X,Y
0,Alberton,Government,Primary,Wellington (S),146.66660,-38.61771
1,Allansford,Government,Primary,Warrnambool (C),142.59039,-38.38628
2,Avoca,Government,Primary,Pyrenees (S),143.47565,-37.08450
3,Avenel,Government,Primary,Strathbogie (S),145.23472,-36.90137
4,Warrandyte,Government,Primary,Manningham (C),145.21398,-37.74268


In [51]:
#Renaming the location coordinate columns 
school_data = school_data.rename(columns={"X":"Lattitude","Y":"Longtitude" })


In [52]:
# Exploring the  dataset to identify if there are any missing values 
print(f"Number of rows in the dataset: {school_data.shape[0]}")
print(f"Number of columns in the dataset: {school_data.shape[1]}\n")
school_data.info()

Number of rows in the dataset: 2302
Number of columns in the dataset: 6

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2302 entries, 0 to 2301
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Address_Town      2302 non-null   object 
 1   Education_Sector  2302 non-null   object 
 2   School_Type       2302 non-null   object 
 3   LGA_Name          2302 non-null   object 
 4   Lattitude         2301 non-null   float64
 5   Longtitude        2301 non-null   float64
dtypes: float64(2), object(4)
memory usage: 108.0+ KB


As it can be seen that only there are 1 missing value in the dataset for the **Lattitude** and **Longtitude** columns, as the main aim of the dataset is to clean the data in a way that we are able to get the number of schools per region (regions that are available in the melbourne-housing dataset), therefore its not necessary to deal with those missing values 

**Calculating number of schools in each suburb** 

In [53]:
# Loading the Melbourne housing data (which contains the list of Melbourne suburbs)
melbourne_housing_data = pd.read_csv(r'C:\Users\DELL\OneDrive\Documents\GitHub\HEM-Housing-ML-Model\Datasets\Melbourne_housing_FULL.csv')

# Keeping only 'Address_Town' and 'School_Type' columns in the school data
school_data = school_data[['Address_Town', 'School_Type']]

# Dropping rows with missing values in 'Address_Town' or 'School_Type' just for precautions because as for now there are no missing values in these columns 
school_data = school_data.dropna(subset=['Address_Town', 'School_Type'])
school_data['Address_Town'] = school_data['Address_Town'].str.title()

# Converting the 'Suburb' column in the Melbourne housing data to lowercase for matching
melbourne_housing_data['Suburb'] = melbourne_housing_data['Suburb'].str.lower()

# Filtering the school data to include only suburbs that are in the Melbourne suburbs list
melbourne_only_school_data = school_data[school_data['Address_Town'].str.lower().isin(melbourne_housing_data['Suburb'])]

# Grouping by 'Address_Town' and calculating the number of schools for each suburb
melbourne_only_school_data = melbourne_only_school_data.groupby('Address_Town').size().reset_index(name='Number_of_Schools')

# Renaming 'Address_Town' to 'Suburb'
melbourne_only_school_data = melbourne_only_school_data.rename(columns={'Address_Town': 'Suburb'})

# Sorting the result in ascending order of suburb names
melbourne_only_school_data = melbourne_only_school_data.sort_values('Suburb')
print(melbourne_only_school_data)


           Suburb  Number_of_Schools
0      Abbotsford                  2
1      Aberfeldie                  2
2    Airport West                  1
3       Albanvale                  1
4     Albert Park                  3
..            ...                ...
304    Wonga Park                  1
305  Wyndham Vale                  4
306     Yallambie                  1
307    Yarra Glen                  1
308    Yarraville                  5

[309 rows x 2 columns]


In [54]:
# Finding suburbs that are in the Melbourne housing dataset but not in the school dataset
suburbs_not_in_schools = [suburb.title() for suburb in melbourne_housing_data['Suburb'].unique() 
                          if suburb not in school_data['Address_Town'].str.lower().unique()]

# Creating a DataFrame for suburbs without school info and set their school count to zero
missing_suburbs_df = pd.DataFrame(suburbs_not_in_schools, columns=['Suburb'])
missing_suburbs_df['Number_of_Schools'] = 0

# Concatenating the original school data with the missing suburbs with zero schools
final_school_data = pd.concat([melbourne_only_school_data, missing_suburbs_df])

# Ensuring the dataset has unique suburbs only by dropping duplicates
final_school_data = final_school_data.drop_duplicates(subset=['Suburb'])
final_school_data = final_school_data.sort_values('Suburb').reset_index(drop=True)

final_school_data

,Suburb,Number_of_Schools
0,Abbotsford,2
1,Aberfeldie,2
2,Airport West,1
3,Albanvale,1
4,Albert Park,3
...,...,...
344,Wonga Park,1
345,Wyndham Vale,4
346,Yallambie,1
347,Yarra Glen,1


**Calculating number of schools in each Region** 

In [58]:
#Removing any missing values in the 'Regionname' and 'Suburb' columns from the housing dataset
melbourne_housing_data = melbourne_housing_data.dropna(subset=['Regionname', 'Suburb'])

#Ensuring the suburb names are in the same format in both datasets
melbourne_housing_data['Suburb'] = melbourne_housing_data['Suburb'].str.title()
final_school_data['Suburb'] = final_school_data['Suburb'].str.title()

# Merging the housing data with the school data to get regions along with school counts
merged_data = pd.merge(final_school_data, melbourne_housing_data[['Suburb', 'Regionname']],on='Suburb', how='left')
merged_data = merged_data.drop_duplicates(subset=['Suburb'])

#Grouping by 'Regionname' and calculate the total number of schools per region by summing school counts by suburb
schools_per_region = merged_data.groupby('Regionname')['Number_of_Schools'].sum().reset_index()
schools_per_region = schools_per_region.rename(columns={'Number_of_Schools': 'Total_Schools_per_Region'})

schools_per_region


,Regionname,Total_Schools_per_Region
0,Eastern Metropolitan,202
1,Eastern Victoria,95
2,Northern Metropolitan,239
3,Northern Victoria,36
4,South-Eastern Metropolitan,185
5,Southern Metropolitan,191
6,Western Metropolitan,229
7,Western Victoria,21


In [ ]:
#Ensuring the summing has been performed accurately 
# Calculating the total number of schools in schools_per_region (summed by region)
total_schools_by_region = schools_per_region['Total_Schools_per_Region'].sum()

# Calculating the total number of schools in final_school_data (summed by suburb)
total_schools_by_suburb = final_school_data['Number_of_Schools'].sum()

print(f"Total schools by region: {total_schools_by_region}")
print(f"Total schools by suburb: {total_schools_by_suburb}")

if total_schools_by_region == total_schools_by_suburb:
    print("The totals match!")
else:
    print("The totals do not match!")

In [56]:
# Saving the final dataset as a CSV file
final_school_data.to_csv(r'C:\Users\DELL\OneDrive\Documents\GitHub\HEM-Housing-ML-Model\clean-datasets\suburb_school_data.csv', index=False)
schools_per_region.to_csv(r'C:\Users\DELL\OneDrive\Documents\GitHub\HEM-Housing-ML-Model\clean-datasets\region_school_data.csv', index=False)